In [ ]:
import carla
import tkinter as tk
from tkinter import ttk

# Funzione per connettersi al server CARLA
def connect_to_carla():
    client = carla.Client('localhost', 2000)
    client.set_timeout(10.0)
    world = client.get_world()
    return client, world

# Funzione per ottenere i punti di spawn con limite
def get_limited_spawn_points(world, lim=50):
    spawn_points = world.get_map().get_spawn_points()
    return spawn_points[:lim]  # Limita il numero di punti di spawn

# Funzione per aggiornare la telecamera sul punto selezionato e stampare "PUNTO DI PARTENZA"
def set_camera_on_spawn(world, spawn_point):
    spectator = world.get_spectator()
    spectator.set_transform(spawn_point)
    # Visualizza "PUNTO DI PARTENZA" sulla mappa
    world.debug.draw_string(spawn_point.location, "PUNTO DI PARTENZA", draw_shadow=False,
                            color=carla.Color(255, 0, 0), life_time=10.0, persistent_lines=True)

# Funzione per ottenere i waypoints entro 1 km dal punto di partenza
def get_nearby_waypoints(world, start_point, lim=50, distance=1000):
    map = world.get_map()
    waypoints = map.generate_waypoints(2.0)  # Genera waypoints con passo di 2 metri
    nearby_waypoints = [
        wp for wp in waypoints if wp.transform.location.distance(start_point.location) <= distance
    ]
    return nearby_waypoints[:lim]  # Limita il numero di waypoints

# Funzione per mostrare "PUNTO DI ARRIVO" sul waypoint selezionato
def show_destination_point(world, waypoint):
    # Visualizza "PUNTO DI ARRIVO" sulla mappa
    world.debug.draw_string(waypoint.transform.location, "PUNTO DI ARRIVO", draw_shadow=False,
                            color=carla.Color(255, 0, 0), life_time=10.0, persistent_lines=True)

# Funzione per spawnare un veicolo
def spawn_vehicle(world, blueprint_library, spawn_point):
    blueprint = blueprint_library.filter('vehicle.*')[0]  # Scegli un veicolo qualsiasi
    vehicle = world.try_spawn_actor(blueprint, spawn_point)
    if vehicle is not None:
        print(f"Veicolo spawnato: {vehicle.type_id}")
    else:
        print("Errore nel spawn del veicolo")

# Funzione per aggiornare i waypoints nel menu a tendina "Destinazione"
def update_waypoints_menu(world, start_point, menu, lim=50, distance=1000):
    waypoints = get_nearby_waypoints(world, start_point, lim=lim, distance=distance)
    waypoint_dict.clear()  # Pulisci il dizionario dei waypoint
    menu['menu'].delete(0, 'end')  # Rimuovi le opzioni precedenti
    for i, waypoint in enumerate(waypoints):
        wp_label = f"WP{i+1}"
        waypoint_dict[wp_label] = waypoint  # Aggiungi il waypoint al dizionario
        menu['menu'].add_command(label=wp_label, command=lambda wp=waypoint: display_distance(start_point, wp))
    if not waypoints:
        menu['menu'].add_command(label="Nessun waypoint", command=lambda: display_distance(start_point, None))

# Funzione per calcolare e visualizzare la distanza tra il punto di partenza e il waypoint
def display_distance(start_point, waypoint):
    if waypoint:
        distance = start_point.location.distance(waypoint.transform.location)
        distance_label.config(text=f"Distanza: {distance:.2f} m")  # Aggiorna l'etichetta con la distanza
        show_destination_point(world, waypoint)  # Mostra "PUNTO DI ARRIVO"
    else:
        distance_label.config(text="Distanza: N/A")  # Nessun waypoint selezionato

# Funzione per il bottone wip1
def wip1_function():
    print("Hello World")

# Funzione per il bottone wip2
def wip2_function():
    print("Hello World")

# Connessione a CARLA
client, world = connect_to_carla()

# Limite per i valori nei menu a tendina
lim = 50
spawn_points = get_limited_spawn_points(world, lim=lim)

# Dizionario per mappare i nomi dei waypoint ai waypoint reali
waypoint_dict = {}

# Creazione della finestra principale con Tkinter
root = tk.Tk()
root.title("CARLA Simulator Interface")
root.geometry('600x400')  # Aumentato la larghezza per far posto ai bottoni

# Etichetta per il menu "Punto di partenza"
start_label = tk.Label(root, text="Punto di partenza")
start_label.grid(row=0, column=0, pady=(10, 0))

# Frame per il menu "Punto di partenza" con bordo
spawn_frame = tk.Frame(root, highlightbackground="black", highlightthickness=2)
spawn_frame.grid(row=1, column=0, pady=5)

# Menu a tendina per selezionare il punto di spawn (SP1, SP2, ...)
selected_spawn = tk.StringVar()
spawn_menu = ttk.OptionMenu(spawn_frame, selected_spawn, "Seleziona Spawn", *[f"SP{i+1}" for i in range(min(len(spawn_points), lim))])
spawn_menu.pack()

# Bottone per impostare il punto di partenza, ora con bordi visibili
spawn_button = tk.Button(root, text="Imposta Punto di Partenza", command=lambda: set_spawn(),
                         highlightthickness=2, highlightbackground="black")
spawn_button.grid(row=1, column=1, padx=5, pady=5)  # Stesso row, ma colonna diversa

# Etichetta per il menu "Punto di arrivo"
destination_label = tk.Label(root, text="Punto di arrivo")
destination_label.grid(row=2, column=0, pady=(10, 0))

# Frame per il menu "Punto di arrivo" con bordo
destination_frame = tk.Frame(root, highlightbackground="black", highlightthickness=2)
destination_frame.grid(row=3, column=0, pady=5)

# Menu a tendina per selezionare la destinazione (WP1, WP2, ...)
selected_destination = tk.StringVar()
destination_menu = ttk.OptionMenu(destination_frame, selected_destination, "Seleziona Waypoint")
destination_menu.pack()

# Etichetta per visualizzare la distanza
distance_label = tk.Label(root, text="Distanza: N/A")
distance_label.grid(row=3, column=1, padx=5)  # Posizionata accanto al menu "Punto di arrivo"

# Bottone per spawnare il veicolo, con bordi visibili
vehicle_button = tk.Button(root, text="Spawn Veicolo", command=lambda: spawn_vehicle_button(),
                           highlightthickness=2, highlightbackground="black")
vehicle_button.grid(row=4, column=0, pady=5)

# Bottone WIP1, con bordi visibili
wip1_button = tk.Button(root, text="WIP1", command=wip1_function,
                        highlightthickness=2, highlightbackground="black")
wip1_button.grid(row=5, column=0, pady=5)

# Bottone WIP2, con bordi visibili
wip2_button = tk.Button(root, text="WIP2", command=wip2_function,
                        highlightthickness=2, highlightbackground="black")
wip2_button.grid(row=6, column=0, pady=5)

# Funzione per impostare il punto di partenza e aggiornare i waypoints
def set_spawn():
    selected_value = selected_spawn.get()
    if selected_value.startswith("SP"):
        selected_index = int(selected_value.replace("SP", "")) - 1
        spawn_point = spawn_points[selected_index]
        set_camera_on_spawn(world, spawn_point)  # Centra la telecamera e mostra "PUNTO DI PARTENZA"
        update_waypoints_menu(world, spawn_point, destination_menu, lim=lim)  # Aggiorna waypoints
    else:
        print("Seleziona un punto di partenza valido.")

# Funzione per spawnare il veicolo in base alla selezione
def spawn_vehicle_button():
    selected_value = selected_spawn.get()
    if selected_value.startswith("SP"):
        selected_index = int(selected_value.replace("SP", "")) - 1
        spawn_vehicle(world, world.get_blueprint_library(), spawn_points[selected_index])
    else:
        print("Seleziona un punto di partenza valido.")

# Configurazione della spaziatura
for widget in root.winfo_children():
    widget.grid_configure(padx=10)

# Avvio dell'interfaccia
root.mainloop()


Veicolo spawnato: vehicle.audi.a2
